Copyright 2024 Netherlands eScience Center and CML, Leiden University
Licensed under the Apache License, version 2.0. See LICENSE for details.

NOTES:
This script allows to estimate the material composition (MC) of products using real product datasets in ecoinvent database.

See paper: Amatuni, L., Steubing, B., Heijungs, R., Yamamoto, T., & Mogollón, J. M. Deriving material composition of products using life cycle inventory databases. Journal of Industrial Ecology. https://doi.org/10.1111/jiec.13538

We highly recommend visiting this paper first to gain a solid understanding of the implementation of this product composition estimation algorithm. We suggest first reviewing a simpler and more universal script "pmc algorithm - general.py" for detailed comments for each code line. This script is an extension of that general script tailoring it to perform on the ecoinvent database. 
The comments under this script will be rather limited. 

REQUIREMENTS: 
- To run this script in Spyder or VS Code you need first to prepare a separate conda environment with an installed Brightway 2 (not 2.5!) framework and (optionally) the Activity Browser GUI on top.
Installing Activity Browser in a separate conda environment as instructed on the AB website will install Brightway and all the packages needed, so this is a good starting point.  
- Basic understanding of Python and Brightway is required to be able to replicate this code for your own products/materials/LCI database of interest
- LCI database: ecoinvent 3.10 cutoff database and the 'biosphere3' database available in your Brigthway project(see constants section for proper linking)

#Concepts to understand
**Material flows hierarchy defined in the paper:**
MF - material footprint; 
MC - material composition; 
NIMF - non-incorporated material footprint.
![MF vs MC](/visuals/mf-and-mc.jpg)

---

**Consider a simplified example of a supply chain of laptop manufacturing:**
![Supply chain](/visuals/supply-chain.jpg)

---

**To estimate the material composition, we want to sum up only the copper amount that actually ends up in the laptop and filter out the rest of the inputs:**
![Filtering](/visuals/filtering.jpg)

---

**The following major steps are part of our approach:**
![Steps](/visuals/steps.jpg)


#Imports
We start with importing all the nesseccary libraries:

In [1]:
from brightway2 import *
from functools import cmp_to_key
import brightway2 as bw
import json
import csv
import sys
import os

#Constants
Then, we define constant parameters used in the script:

In [3]:
PROJECT_NAME = 'default' #name of your Brightway project
DB_NAME = 'ecoinvent-3.10-cutoff' #name of your LCI database in your Brightway project 
BIO_DB_NAME = 'biosphere3' #name of your bioflows database in your Brightway project 
product = 'computer production, laptop' #the product that we want to explore for its materil content; the name should come from the LCI database you will use
prod_wght = 3.15 #kg per product ideally specified in your LCI database (comes from Ecoinvent 3.6 in our case)
METHOD_KEY = ('ReCiPe 2016 v1.03, endpoint (H)', 'natural resources', 'material resources: metals/minerals') #selected method from the list of impact methods; in practice arbitrary as it does not impact the resulting inventory/supply vectors but is needed to run the lca.lci() command
Cu = "Copper"
Al = "Aluminium"
Ta = "Tantalum"
BIO_MAT_LIST = [Cu, Al, Ta] #the natural materials of interest (the appropariate flows in the biospere database will be selected later on based on this names). Each name should start with the capital letter (see conventional names of materials/metals in the biosphere3 database)
FU = 1 #amount of the product of interest (functional unit), e.g. one unit laptop
#auxiliary:
KEY_index = 1 #index of the actual activity/bioflow key in a conventional tuple key like (db, key)
FLOAT_RND = 5 #how many digits left after floating point 

#Project and databases set up
It is assumed that you already have Brightway project, ecoinvent and biosphere databases.
If not, it is easy to set them up and download them using Activity Browser.
You can also use `bw2setup()` to do that in Brightway. 

###Here, we open our existing Brightway project with LCI and biosphere databases: 

In [4]:
projects.set_current(PROJECT_NAME)
db  = bw.Database(DB_NAME)
bio = bw.Database(BIO_DB_NAME)

###Get the directory where the script is located and set the current working directory to the script's directory:

In [5]:
script_dir = os.path.dirname(os.path.abspath(sys.argv[0]))
os.chdir(script_dir)

#Material selection 
Here, we will link specific materials that we are interested in to their production activities in our LCI database.
Hint: we use markets instead of production activities as they conveniently include all the regional prod. activities and there's no need to list them separately.

The so-called *material dictionary* is manually assembled by copying corresponding activities' keys (db name, key) from Acivity Browser and grouping them under a common name: 

In [6]:
materials_dict_cutoff310 = {
        "Metals":
            {Cu: [(DB_NAME, 'bc9651dcc7c9e1666633deebe9cc51ba')],
             Al: [(DB_NAME, 'e540cdb4add7b620e2d2d64a3abb418d'), (DB_NAME, '56a38ae7dd7648bab5997fab280bbf46')] #market for aluminium, cast alloy + market for aluminium, wrought alloy 
            }
}

For plastics, however, we had to automatically scan the whole ecoinvent for various plastic types and prepare a big list of activities representing various plastic types. It would be almost impossible to do that manually. 
In this script, we just attach that JSON file that we have prepared in advance for plastics to our *material dictionary*.
We also combine all the plastic types under a common material *Plastics* to save screen space when we output results later on. 

In [ ]:
#import plastics dictionary from JSON
with open('dict_gen/plastics_dict_ecoinvent-3.10-cutoff_m.json', 'r') as fp:
    materials_dict_cutoff310["Plastics"] = json.load(fp) 

In [ ]:
#flatten all tuples under 'Plastics' into a single list 
combined_plastics = [item for sublist in materials_dict_cutoff310["Plastics"].values() for item in sublist]
materials_dict_cutoff310["Plastics"] = {"Total": combined_plastics}

#Avoid list
This list specifies types of exchanges that usually do not become part of the product that inputs them in ecoinvent. 
Scanning for such keywords in the LCI database, allows to distinguish *incorporated* and *non-incorporated* material flows. 

In [7]:
avoid_activities = ["treatment", "water", "waste", "container", "box", "packaging", "foam", "electricity", "factory", "adapter", "oxidation", "construction", "heat", "facility", "gas", "freight", "mine", "infrastructure", "conveyor", "road", "building", "used", "maintenance", "transport", "moulding", "mold", "wastewater", "steam", "scrap", "converter"]

#Functions
Here, we define various function that can be used while estimating teh material of products using LCI databases.

In [15]:
def activity_by_name(name, db): #return first activity dataset based on name keyword
    candidates = [x for x in db if name in x['name']]
    candidates = sorted(candidates, key=cmp_to_key(lambda item1, item2: len(item1['name']) - len(item2['name'])))  #shortest name is the best match
    return candidates[0]

def activity_by_key(key, db): # key = tuple(db, key) -> activity (dataset) in db
    return db.get(key[1])

#List all intermediate (technosphere) flows (activities) in the resulting supply-array (see the Paper) that is stored in the reuslting 'lca' object
def list_techno_inventory(lca):
    print("\u25A0 Supply array: ")
    for k in lca.activity_dict:
        print(activity_by_key(k, db)["name"], ": ", lca.supply_array[lca.activity_dict[k]])
    print()

#For the product of interest from the database 'db' list incorporation parameters for all inputs of its production process
def product_inputs(prod, db):
    act = activity_by_name(prod, db)
    for exc in act.technosphere():
        try:
            print(bw.get_activity(exc["input"])._document.product, exc['incorporated'])
        except: 
            print('Error: Not all of the exchanges in your LCI database have the incorporation parameter assigned!')
            sys.exit(1) 

#Resets inc. parameter in the original db (~30 min)
def db_inc_filter(db, avoid_activities):
    i = 0
    prt = -1
    for act in db:
        # update the bar
        i += 1
        pr = int(100*i/len(db))
        if  (pr != prt):
            b = "\rAssigning material incorporation parameters to all exchanges in the " + db.name + " database: " + str(pr) + "%"
            print (b, end="\r")
            prt = pr
        for exc in act.technosphere():
            avoid = False
            in_act = bw.get_activity(exc["input"])
            exc_name = in_act._document.product #product name; same as exc["name"] but works for manual db
            for word in avoid_activities:
                if word in exc_name: #name of the product of the exchange
                    avoid = True
                    break
            exc['incorporated'] = 0.0 if avoid else 1.0
            exc.save()
    print('\n')

#Restores all the incorporation parameters in the database back to 1
def db_inc_reset(db): 
    i = 0
    prt = -1
    for act in db:
        # update the bar
        i += 1
        pr = int(100*i/len(db))
        if  (pr != prt):
            b = "\rApplying full (1.0) incorporation parameters to the activities: " + str(pr) + "%"
            print (b, end="\r")
            prt = pr
        for exc in act.technosphere():
            exc['incorporated'] = 1.0
            exc.save()
    print('\n')  
            
#Edits lca techn. matrix (exclude non-incorporative exc) based on incorporation parameter in the db database
def lca_exclude_noninc(db, lca): 
    i = 0
    prt = -1
    print('\n')
    for act in db:
        # update the bar
        i += 1
        pr = int(100*i/len(db))
        if  (pr != prt):
            b = "\rExcluding the non-incorporated materials from the technosphere matrix: " + str(pr) + "%"
            print (b, end="\r")
            prt = pr
        for exc in act.technosphere():
            try:
                inc = exc['incorporated']
            except:
                inc = 1 #if the incorporation parameter was not entered in ab or set by db_inc_filter() previously
                print('Error: missing incorporation parameter in the LCI database detected! -> assigned to 1')
            if  inc < 1:
                row = lca.activity_dict[exc["input"]]
                col = lca.activity_dict[act.key]
                lca.technosphere_matrix[row, col] *= inc
    return lca

#Creates an LCA object based on the reference product 'act' in the database 'db' and the bioflow 'material_bioflow' of interest     
def LCA_create(act, FU): 
    functional_unit = {act: FU}
    return bw.LCA(functional_unit, METHOD_KEY)

#Outputs the material bioflows of interest, 'mat_list' from the resulting inventory vector in the 'lca' object 
# This is basically the MC or MF of a product, depending if material filtering was applied before. 
def materials_inv(mat_list, lca): 
    for flow_index, amount in enumerate(lca.inventory.sum(axis=1).flat): # lca.inventory.sum(axis=1).flat gives you the summed inventory for each biosphere flow
        flow_key = list(lca.biosphere_dict.items())[flow_index][0][KEY_index] #obtain key of each bioflow (in the resulting 'inventory') based on the 'biosphere_dict' that lists the keys of the resulting elementary flows 
        flow_name = bio.get(flow_key)['name'] #obtain name of each bioflow using its key based on the 'bio' database that contains the names of all elementary flows
        if flow_name in mat_list:
            print(f'{flow_name}: {round(amount, FLOAT_RND)} kg OR {round(amount/prod_wght * 100, FLOAT_RND)} %') 
    print('\n')
    return 0

#Given predefined 'materials_dict' (see above), aggregates and prints 
# resulting material fllows (MC or MF of a product, depending if filtering was applied) 
# using the 'supply_array' from the resulting 'lca' object
def materials_sup(materials_dict, lca):
    try:
        for material_group in materials_dict:
            gr_sum = 0
            for material in materials_dict[material_group]:
                mat_sum = 0
                for act_key in materials_dict[material_group][material]:
                    act_key = tuple(act_key) #this fix to is needed as the keys from the fp file are given as a list [db_name,key]
                    mat_sum += lca.supply_array[lca.activity_dict[act_key]]
                print(f'{material} : {round(mat_sum, FLOAT_RND)} kg OR {round(mat_sum/prod_wght * 100, FLOAT_RND)} %')
                gr_sum += mat_sum
            print(f'> {material_group} total : {round(gr_sum, FLOAT_RND)} kg OR {round(gr_sum/prod_wght * 100, FLOAT_RND)} %')
    except Exception as e: 
        print(f'\nError: most likely you used a wrong key in your material dictionary that does not properly link to the activity in the LCI database! \nCheck the {e}')
        sys.exit(1) 
    return 0

def db_amount_save(db): #save all the original amounts of the exchanges
    for act in db:
        for exc in act.technosphere():
            exc['amount_save'] = exc['amount']
            exc.save()
            
def db_amount_restore(db): #restore all the original amounts of the exchanges
    for act in db:
        for exc in act.technosphere():
            exc['amount'] = exc['amount_save']
            exc.save()
            
def db_inc_to_amounts(db): #adjust all the amounts of the exchanges based on the incorporation
    for act in db:
        for exc in act.technosphere():
            exc['amount'] = exc['amount_save'] * exc['incorporated']
            exc.save()    

def db_to_csv(db): #save datasets (name, key) into the csv file
    #generate a list of act. names and their keys
    list = [['name','key']]
    for act in db:
        list.append([act["name"], act.key[1]])
    #write into csv
    with open(db.name+'.csv', 'w') as f:
        writer = csv.writer(f, delimiter='|', lineterminator="\n")
        writer.writerows(list)
    return str(db) + " saved into " + db.name + '.csv'

---
#Execution

###Assigning material incorporation paramter to each exchange in your LCI database:
Runs through ecoinvent activities and assign material incorporation parameter (from 0 to 1) 
to each exchange based on the list of keywords in the 'avoid_activities' list of keywords.

This **should be done only once** and the edited LCI database is saved in your broject. 
Takes about 15-30 min. 

In [ ]:
db_inc_filter(db, avoid_activities) 

This step could also be done manually in Activity Browser, if the corresponding additional column woudl be added:
![AB](/visuals/ab-incorporation.png)

###Estimating the material footprint and material composition:
Here, for we calculate product MF (material footprint) and MC (material composition).

First, we find the corresponding production activity of the product of interest in our LCI database, and create a Brightway LCA object based on that product and its amount (funcional unit):

In [9]:
act = activity_by_name(product, db)
lca = LCA_create(act, FU)
lca.lci() #creates technosphere

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Then, we output the material footprint of each material of interest for our product.
First, based on inventory vector and then based on supply array:

In [16]:
print("\n>>> BEFORE filtering:\n")  

print(f'\u25A0 Material footprint, MF (based on inventory vector) in {FU} {act}:') 
materials_inv(BIO_MAT_LIST, lca)
    
print("\u25A0 Material footprint, MF (based on supply array):")
materials_sup(materials_dict_cutoff310, lca) 


>>> BEFORE filtering:

■ Material footprint, MF (based on inventory vector) in 1 'computer production, laptop' (unit, GLO, None):
Tantalum: 0.00019 kg OR 0.00604 %
Aluminium: 0.48225 kg OR 15.30958 %
Copper: 0.36106 kg OR 11.46236 %


■ Material footprint, MF (based on supply array):
Copper : 0.31864 kg OR 10.11559 %
Aluminium : 0.62168 kg OR 19.73588 %
> Metals total : 0.94032 kg OR 29.85146 %


0

Then, we edit the technosphere matrix based on the *incorporation parameter* pre-assigned to exchanges in our LCI database:

In [ ]:
lca_exclude_noninc(db, lca) 
lca.lci_calculation()

Fianlly, we output the total material content of our product (first, based on inventory vector and then based on supply array):

In [17]:
print("\n>>> AFTER filtering:\n")
    
print(f'\u25A0 Material composition, MC (based on inventory vector) in {FU} {act}:')
materials_inv(BIO_MAT_LIST, lca)
    
print("\u25A0 Material composition, MC (based on supply array):")
materials_sup(materials_dict_cutoff310, lca)


>>> AFTER filtering:

■ Material composition, MC (based on inventory vector) in 1 'computer production, laptop' (unit, GLO, None):
Tantalum: 0.00019 kg OR 0.00604 %
Aluminium: 0.48225 kg OR 15.30958 %
Copper: 0.36106 kg OR 11.46236 %


■ Material composition, MC (based on supply array):
Copper : 0.31864 kg OR 10.11559 %
Aluminium : 0.62168 kg OR 19.73588 %
> Metals total : 0.94032 kg OR 29.85146 %


0

In the result, we are able to accuratelly estimate the material composition of products:
![Results](/visuals/results.jpg)